# Create dataframes for image loader

In [ ]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import os


import torch
from skimage import io, transform
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from torchvision import datasets
from torchvision.transforms import ToTensor
import cv2
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torch.optim import Adam, SGD
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split, TensorDataset, Dataset, WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from torchvision import models
from torchmetrics.classification import BinaryF1Score, BinaryPrecision, BinaryRecall, BinaryAccuracy, BinaryROC, BinaryAUROC
import PIL
from PIL import Image

# visualisation
import seaborn as sns

# helpers
from tqdm import tqdm
import time
import copy
import gc
from enum import Enum

import warnings
warnings.filterwarnings('ignore')

metadata_list=['mass_case_description_test_set','mass_case_description_train_set']
#'mass_case_description_test_set','mass_case_description_train_set',
df_list=[]
for j in range(len(metadata_list)):
    print (metadata_list[j])
    df=pd.read_csv('/home/mbadhan/Desktop/mberghouse/'+metadata_list[j]+'.csv')
    print (len(df))
    
    fname=[]

    df=df.rename(columns={"file path": "filename","pathology":"class", 'image file path':'filename','cropped image file path':'patch_filename'})
###Remove multiple-counted images for whole image
    for k in range(len(df)):
        fname.append(df['filename'].loc[k])
        if k>0:
            if fname[k] == fname[k-1]:
                df.drop(k, inplace=True)
    print ('df length after removal of repeats: ', len(df))
    
    
    nan_count=0

    for i in range(len(df)):
### For whole images
        df.filename.iloc[i]='/home/mbadhan/Desktop/mberghouse/CBIS-DDSM-preprocessed/600x1000_v6/'+df.filename.iloc[i].rsplit('/',3)[0]+'/1-1.png'
        #df.filename[i]='F:/CBIS_DDSM_redo/manifest-ZkhPvrLo5216730872708713142/CBIS-DDSM-png/'+df.filename[i].rsplit('/',3)[0]+'/1-1.png'
        if 'MALIGNANT' in df['class'].iloc[i]:
            df['class'].iloc[i]=1
        else:
            df['class'].iloc[i]=0
    df_list.append(df)
    
### For patches
        ##i=i-nan_count
       ## df.filename.iloc[i]='C:/Users/marcb/Desktop/CBIS-DDSM-preprocessed/v4_3d/'+df.filename.iloc[i].rsplit('/',3)[0]+'/1-1.jpg'
        ##df.filename[i]='F:/CBIS_DDSM_redo/manifest-ZkhPvrLo5216730872708713142/CBIS-DDSM-png/'+df.filename[i].rsplit('/',3)[0]+'/1-1.png'
### For Masses
#         try:
#             path1 = 'F:/CBIS_DDSM/manifest-ZkhPvrLo5216730872708713142/CBIS-DDSM-mask/'+df.patch_filename.iloc[i].rsplit('/',3)[0]
#             folder1=os.listdir(path1)[0]
#             folder2=os.listdir(os.path.join(path1,folder1))[0]
#             img = os.listdir(os.path.join(path1,folder1,folder2))
#             im0_path=os.path.join(path1,folder1,folder2,img[0])
#             im0=cv2.imread(im0_path)
#             if len(img)>1:
#                 if np.median(im0)==0:
#                     img_path = os.path.join(path1,folder1,folder2,img[1])
#                 else:
#                     img_path = im0_path
#             else:
#                 if np.median(im0)==0:
#                     img_path = np.nan
#                     nan_count = nan_count+1
#                 else:
#                     img_path = im0_path
#             df.patch_filename.iloc[i]=img_path
#         except:
#             path1 = 'F:/CBIS_DDSM/manifest-ZkhPvrLo5216730872708713142/cropped/'+df.patch_filename.iloc[i].rsplit('/',3)[0]
#             folder1=os.listdir(path1)[0]
#             folder2=os.listdir(os.path.join(path1,folder1))[0]
#             img = os.listdir(os.path.join(path1,folder1,folder2))
#             im0_path=os.path.join(path1,folder1,folder2,img[0])
#             im0=cv2.imread(im0_path)
#             if len(img)>1:               
#                 if np.median(im0)==0:
#                     img_path = os.path.join(path1,folder1,folder2,img[1])
#                 else:
#                     img_path = im0_path
#             else:
#                 if np.median(im0)==0:
#                     img_path = np.nan
#                     nan_count = nan_count+1
#                 else:
#                     img_path = im0_path
#             df.patch_filename.iloc[i]=img_path
#             #df.dropna(inplace=True)
        
#         if 'MALIGNANT' in df['class'].iloc[i]:
#             df['class'].iloc[i]=1
#         else:
#             df['class'].iloc[i]=0
#     df.dropna(inplace=True)
#     df_list.append(df)
#df_list
# df_mass_test=df_list[0]
# df_mass_train=df_list[1]
df_calc_test=df_list[0]
df_calc_train=df_list[1]

# filenames=[]
# labels=[]
# for i in range(len(df_mass_train)):
#     filenames.append(df_mass_train.filename.iloc[i])
#     labels.append(df_mass_train['class'].iloc[i])

# filenames_test=[]
# labels_test=[]
# for i in range(len(df_mass_test)):
#     filenames_test.append(df_mass_test.filename.iloc[i])
#     labels_test.append(df_mass_test['class'].iloc[i])
    
# filenames_calc=[]
# labels_calc=[]
# for i in range(len(df_calc_train)):
#     filenames_calc.append(df_calc_train.filename.iloc[i])
#     labels_calc.append(df_calc_train['class'].iloc[i])

# filenames_test_calc=[]
# labels_test_calc=[]
# for i in range(len(df_calc_test)):
#     filenames_test_calc.append(df_calc_test.filename.iloc[i])
#     labels_test_calc.append(df_calc_test['class'].iloc[i])

In [ ]:
filenames_calc=[]
labels_calc=[]
for i in range(len(df_calc_train)):
    filenames_calc.append(df_calc_train.filename.iloc[i])
    labels_calc.append(df_calc_train['class'].iloc[i])

filenames_test_calc=[]
labels_test_calc=[]
for i in range(len(df_calc_test)):
    filenames_test_calc.append(df_calc_test.filename.iloc[i])
    labels_test_calc.append(df_calc_test['class'].iloc[i])

In [ ]:

import torchvision
import numpy.matlib as np_mlb
#torchvision.disable_beta_transforms_warning()
#import torchvision.transforms.v2 as transforms

#plt.ion()   # interactive mode

##########################################
##### HERE ARE THE AUGMENTATIONS!!! ######
##########################################
# affine_transfomer = T.RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75))

augmentator = transforms.Compose([
    # input for augmentator is always PIL image
    transforms.ToPILImage(),
    
    transforms.ColorJitter(brightness=(.6,1.2)),
#     transforms.Pad(padding=(12,3)),
    #torchvision.models.EfficientNet_V2_S_Weights.IMAGENET1K_V1.transforms(),
#     transforms.RandomAdjustSharpness(1.5, p=0.5),
#     transforms.RandomPhotometricDistort(),
    #transforms.RandomZoomOut(fill = 0, side_range = (1, 1.3)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(12),
#     transforms.Resize(size=(500,300)),
    #transforms.RandomAffine(degrees=(0,10),translate=(0.01, 0.02),  shear=(.5,.5)),
    #transforms.GaussianBlur(kernel_size=(3, 3), sigma=(.5)),
#     transforms.RandomEqualize(p=.1),
#    transforms.RandomAutocontrast(),
    transforms.ToTensor(),
    #transforms.ConvertImageDtype(torch.float32),# return it as a tensor and transforms it to [0, 1]
])

small_aug = transforms.Compose([
    # input for augmentator is always PIL image
    transforms.ToPILImage(),
    #transforms.Pad(10),
    #torchvision.models.EfficientNet_V2_S_Weights.IMAGENET1K_V1.transforms()
    transforms.ToTensor(),
    #transforms.v2.ConvertImageDtype(torch.float32)# return it as a tensor and transforms it to [0, 1]
])

def show_landmarks(image, label):
    """Show image with landmarks"""
    plt.imshow(image)
    if label==0:
        classif='Benign'
    else:
        classif='Malignant'
    print ('Class: ',classif)
    plt.pause(0.001)  # pause a bit so that plots are updated

class CBISDataset(Dataset):
    """CBIS-DDSM dataset."""

    def __init__(self, labels, filenames, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels
        self.filenames = filenames
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        fname = self.filenames[idx]
        image = cv2.imread(fname)
#         image = cv2.resize(image, (344,344))
#         image = (((image-np.min(image))/(np.max(image)-np.min(image)))*255).astype('uint8')
#         clahe = cv2.createCLAHE(clipLimit = 2,tileGridSize=(12,12))
#         image = clahe.apply(image[:,:,0])
#         image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
       # image = torchvision.io.read_image(fname)#, torchvision.io.ImageReadMode.RGB)
       # image = image.repeat([3, 1, 1])
        label = self.labels[idx]    
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# filenames_calc.pop(375)
# filenames_calc.pop(494)
# filenames_calc.pop(705)
# filenames_calc.pop(928)
# filenames_calc.pop(928)
# filenames_calc.pop(928)
# labels_calc.pop(375)
# labels_calc.pop(494)
# labels_calc.pop(705)
# labels_calc.pop(928)
# labels_calc.pop(928)
# labels_calc.pop(928)

In [ ]:
# filenames_calc.pop(520)
# filenames_calc.pop(520)
# filenames_calc.pop(838)
# filenames_calc.pop(1107)
# filenames_calc.pop(1107)

# labels_calc.pop(520)
# labels_calc.pop(520)
# labels_calc.pop(838)
# labels_calc.pop(1107)
# labels_calc.pop(1107)

In [ ]:
dataset = CBISDataset(labels_calc,filenames_calc,transform=augmentator)

val_pct = 0.1
val_size = int(val_pct * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
test_dataset = CBISDataset(labels_test_calc,filenames_test_calc, transform=small_aug)
val_size =  len(val_dataset)
train_size = len(train_dataset)

In [ ]:
batch_size = 10
#try pin memory = False

# Create data loaders.
#train_dataloader = DataLoader(dataset, batch_size=batch_size,
#                        shuffle=True)#, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size,shuffle=False, pin_memory = True,num_workers=batch_size)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True, pin_memory = True,num_workers=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True, pin_memory = True,num_workers=batch_size)


for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    #img=X
    break
plt.imshow(X[0,0,:,:])

In [ ]:

    
    
# create class for earlystopping
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_loss = np.inf

    def early_stop(self, loss):
        if loss <= self.min_loss:
            self.min_loss = loss
            self.counter = 0
        elif loss > (self.min_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
    
    
def BCELoss_class_weighted(weights):
    """
    weights[0] is weight for class 0 (negative class)
    weights[1] is weight for class 1 (positive class)
    """
    def loss(y_pred, target):
        y_pred = torch.clamp(y_pred,min=1e-7,max=1-1e-7) # for numerical stability
        bce = - weights[1] * target * torch.log(y_pred) - (1 - target) * weights[0] * torch.log(1 - y_pred)
        return torch.mean(bce)

    return loss

def find_optim_thres(fpr, tpr, thresholds):
    optim_thres = thresholds[0]
    inx = 0
    min_dist = 1.0
    for i in range(len(fpr)):
        dist = np.linalg.norm(np.array([0.0, 1.0]) - np.array([fpr[i], tpr[i]]))
        if dist < min_dist:
            min_dist = dist
            optim_thres = thresholds[i]
            inx = i
            
    return optim_thres, inx


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    scheduler1=scheduler[1]
    scheduler=scheduler[0]
    since = time.time()
    metricf1 = BinaryF1Score()
    precision = BinaryPrecision()
    recall = BinaryRecall()
    accuracy = BinaryAccuracy()
    roc = BinaryROC()
    auc = BinaryAUROC()
    best_model_wts = model.state_dict()
    stop_count = 0
    best_f1 = -1.0
    train_metrics = {'loss' : [], 'acc' : [], 'f1': [], 'precision': [], 'recall': [], 'auc': []}
    val_metrics = {'loss' : [], 'acc' : [], 'f1': [], 'precision': [], 'recall': [], 'auc': []}
    # inital threshold for first epoch, it will change afterwards
    threshold = 0.5
    sched_steps=[]
    print('Starting training...')
    print('-' * 20)
    for epoch in range(num_epochs):
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            # empty 'all' tensors for saving
            # for calculating aoc at the end of epoch, and for calculating new threshold
            all_outputs = torch.Tensor([])
            all_labels = torch.Tensor([])
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            n_samples = 0
            n_correct = 0
            running_f1 = 0.0
            # Iterate over data.
            print(f'{phase} for epoch {epoch + 1}')
            for inputs, labels in tqdm(dataloaders[phase]):
                labels = torch.unsqueeze(torch.tensor(labels), 1).to(dtype=torch.float)               
                #labels=torch.tensor(labels)
                inputs = inputs.float()
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = (outputs > threshold).double()
                    #print(all_outputs)
                    #print(outputs)
                    # concatenating all outputs and labels for calculation aoc and new threshold
                    all_outputs = torch.cat((all_outputs, outputs.to('cpu')))
                    all_labels = torch.cat((all_labels, labels.to('cpu')))                  
                    #print(labels)
                    # _, preds = torch.max(outputs, 1)
#                     loss=criterion(model(inputs)[model(inputs)>0], labels[model(inputs)>0])
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                        scheduler1.step()
#                         start=scheduler.state_dict()
                        sched_steps.append(optimizer.param_groups[0]['lr'])
#                         if epoch == 18:
#                             lower=scheduler.state_dict()
#                         if epoch ==40:
# #                             optimizer = torch.optim.NAdam(model.parameters(), lr=1e-5, weight_decay=.1)
# #                             scheduler=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 1600, T_mult=2, eta_min=1e-10, last_epoch=- 1, verbose=False)

#                             for name, param in model.named_parameters():
#                                 param.requires_grad = True
#                         if epoch ==60:
#                             optimizer.param_groups[0]['lr']=1e-5
#                             scheduler.load_state_dict(start)


                running_loss += loss.item()
                # collect any unused memmory
                gc.collect()
                torch.cuda.empty_cache()           
            # statistics
            epoch_loss = running_loss / len(dataloaders[phase])            
            # find true positive and false positive rates for ROC curve
            #print ('outputs: ', all_outputs, 'labels', all_labels)
            all_labels=all_labels.to(dtype=torch.long)
            fpr, tpr, thresholds = roc(all_outputs, all_labels)
            epoch_auc = auc(all_outputs, all_labels)
            # find new threshold
            threshold, _ = find_optim_thres(fpr, tpr, thresholds)
            print(f'New threshold is {threshold}')
            # calculate metrics using new optimized threshold
            epoch_f1 = metricf1(all_outputs > threshold, all_labels)
            epoch_acc = accuracy(all_outputs > threshold, all_labels)
            epoch_precision = precision(all_outputs > threshold, all_labels)
            epoch_recall = recall(all_outputs > threshold, all_labels)
            print(f'{phase} F1 is {epoch_f1}')            
            # save all of the statistics for latter analysis
            if phase == 'train':
                wandb.log({"train_acc": epoch_acc, "train_loss": epoch_loss, "train_f1": epoch_f1, "train_precision": epoch_precision
                         , "train_recall": epoch_recall, "train_auc": epoch_auc})
                train_metrics['loss'].append(epoch_loss)
                train_metrics['acc'].append(epoch_acc)
                train_metrics['f1'].append(epoch_f1)
                train_metrics['precision'].append(epoch_precision)
                train_metrics['recall'].append(epoch_recall)
                train_metrics['auc'].append(epoch_auc)
            else:
                wandb.log({"val_acc": epoch_acc, "val_loss": epoch_loss, "val_f1": epoch_f1, "val_precision": epoch_precision
                         , "val_recall": epoch_recall, "val_auc": epoch_auc})
                val_metrics['loss'].append(epoch_loss)
                val_metrics['acc'].append(epoch_acc)
                val_metrics['f1'].append(epoch_f1)
                val_metrics['precision'].append(epoch_precision)
                val_metrics['recall'].append(epoch_recall)
                val_metrics['auc'].append(epoch_auc)

            # deep copy the model
                if val_metrics['f1'][-1] > best_f1:
                    best_f1 = val_metrics['f1'][-1]
                    best_model_wts = model.state_dict()
                    checkpoint['threshold'] = threshold
                    torch.save(checkpoint, 'checkpoint.pth')
               
        # cant be formated in strin g
        tr_loss, tr_acc, tr_f1, tr_prec, tr_rec, tr_auc = train_metrics['loss'][-1], train_metrics['acc'][-1],  train_metrics['f1'][-1], train_metrics['precision'][-1], train_metrics['recall'][-1], train_metrics['auc'][-1]
        val_loss, val_acc, val_f1, val_prec, val_rec, val_auc = val_metrics['loss'][-1], val_metrics['acc'][-1], val_metrics['f1'][-1], val_metrics['precision'][-1], val_metrics['recall'][-1], val_metrics['auc'][-1]
        lr = optimizer.param_groups[0]['lr']
        print(f'Epoch {epoch + 1}/{num_epochs}, learning rate: {lr}')
        print(f'Train Loss: {tr_loss:.4f}, Train Acc: {tr_acc:.4f}, Train f1: {tr_f1:.4f}, Train Precision: {tr_prec:.4f}, Train Recall: {tr_rec:.4f}, Train AUC: {tr_auc:.4f}')
        print(f'Valitadion Loss: {val_loss:.4f}, Validation Acc: {val_acc:.4f}, Vall f1: {val_f1:.4f}, Val Precision: {val_prec:.4f}, Val Recall: {val_rec:.4f}, Val AUC: {val_auc:.4f}')       
#         if epoch == 0:
#             val_f1_best=val_f1
#         else:
#             if val_f1 > val_f1_best:
#                 val_f1_best=val_f1
#                 stop_count = 0
#             else:
#                 stop_count = stop_count + 1
        
#         if stop_count == 160:
#             break
                
#         if earlystoper.early_stop(val_f1):
#             break       
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val auc: {best_f1:4f}')
    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model, train_metrics, val_metrics, sched_steps

In [ ]:
def test_model(model,criterion,dataloader, threshold=.5):
    device='cuda'
    since = time.time()
    metricf1 = BinaryF1Score()
    precision = BinaryPrecision()
    recall = BinaryRecall()
    accuracy = BinaryAccuracy()
    roc = BinaryROC()
    auc = BinaryAUROC()
    stop_count = 0
    best_f1 = -1.0
    test_metrics = {'loss' : [], 'acc' : [], 'f1': [], 'precision': [], 'recall': [], 'auc': []}
    # inital threshold for first epoch, it will change afterwards
#     threshold = 0.5
    print('Starting testing...')
    # empty 'all' tensors for saving
    all_outputs = torch.Tensor([])
    all_labels = torch.Tensor([])
    model.eval()   # Set model to evaluate mode
    running_loss = 0.0
    n_samples = 0
    n_correct = 0
    running_f1 = 0.0
    # Iterate over data.
    for inputs, labels in tqdm(dataloader):
        labels = torch.unsqueeze(torch.tensor(labels), 1).to(dtype=torch.float)               
        #labels=torch.tensor(labels)
        inputs = inputs.float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.no_grad():
        # zero the parameter gradients
            outputs = model(inputs)
            #preds = (outputs > threshold).double()
            # concatenating all outputs and labels for calculation aoc and new threshold
            all_outputs = torch.cat((all_outputs, outputs.to('cpu')))
            all_labels = torch.cat((all_labels, labels.to('cpu')))                  
            #print(labels)
            # _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
        running_loss += loss.item()
        #n_correct += (preds == labels).sum().item()
        # collect any unused memmory
        gc.collect()
        torch.cuda.empty_cache()  
        
    # statistics
    epoch_loss = running_loss / len(dataloader)            
    # find true positive and false positive rates for ROC curve
    #print ('outputs: ', all_outputs, 'labels', all_labels)
    all_labels=all_labels.to(dtype=torch.long)
    fpr, tpr, thresholds = roc(all_outputs, all_labels)
    epoch_auc = auc(all_outputs, all_labels)
    # find new threshold
    threshold, _ = find_optim_thres(fpr, tpr, thresholds)
    print(f'New threshold is {threshold}')
    # calculate metrics using new optimized threshold
    epoch_f1 = metricf1(all_outputs > threshold, all_labels)
    epoch_acc = accuracy(all_outputs > threshold, all_labels)
    epoch_precision = precision(all_outputs > threshold, all_labels)
    epoch_recall = recall(all_outputs > threshold, all_labels)
    # save all of the statistics for latter analysis
    test_metrics['loss'].append(epoch_loss)
    test_metrics['acc'].append(epoch_acc)
    test_metrics['f1'].append(epoch_f1)
    test_metrics['precision'].append(epoch_precision)
    test_metrics['recall'].append(epoch_recall)
    test_metrics['auc'].append(epoch_auc)               
    time_elapsed = time.time() - since
    wandb.log({"test_acc": epoch_acc, "test_loss": epoch_loss, "test_f1": epoch_f1, "test_precision": epoch_precision
                         , "test_recall": epoch_recall, "test_auc": epoch_auc})
    print(f'Inference complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'F1 Score = : {epoch_f1:4f}')
    print(f'AUC Score = : {epoch_auc:4f}')
    print(f'Acc Score = : {epoch_acc:4f}')
    return test_metrics

# Import Model From Timm and Reset Classifier. SE and CBAM Models are created by changing the source code

In [ ]:



# import timm
# model = timm.create_model('densenet121', pretrained=True)

# num_in_features = model.get_classifier().in_features

# # Replace the existing classifier. It's named: classifier
# model.classifier = nn.Sequential(
#     nn.Dropout(.3),
#     nn.Linear(in_features=num_in_features, out_features=64, bias=False),
#     nn.LeakyReLU(.1,inplace=True),
#     #nn.Dropout(.5),
#     nn.Linear(in_features=64, out_features=1, bias=False),
#     nn.Sigmoid())

# model

In [1]:
# model=my_resnet50(weights="IMAGENET1K_V2")
# for name, param in model.named_parameters():
#     if ('sa' not in name)&('ca' not in name)&('classifier' not in name)&('se' not in name):
#         param.requires_grad = False
#     print(name, param.requires_grad)
import timm
model = timm.create_model('ECAdensenet121', pretrained=True)
#for i in range(3):
    
# model.s1.b1.att.channel.act = nn.LeakyReLU(.1)
# model.s1.b2.att.channel.act = nn.LeakyReLU(.1)
# model.s2.b1.att.channel.act = nn.LeakyReLU(.1)
# model.s2.b2.att.channel.act = nn.LeakyReLU(.1)
# model.s3.b1.att.channel.act = nn.LeakyReLU(.1)
# model.s3.b2.att.channel.act = nn.LeakyReLU(.1)
# model.s4.b1.att.channel.act = nn.LeakyReLU(.1)
#model.s4.b2.att.channel.act = nn.LeakyReLU(.1)
        
#     param.requires_grad=False

# for name, param in model.named_parameters():
#     if 'se' in name:
#         param.requires_grad=True
# num_in_features = model.get_classifier().in_features

# # Replace the existing classifier. It's named: classifier
# model.classifier = nn.Sequential(
#     nn.Dropout(dropout),
#     nn.Linear(in_features=num_in_features, out_features=hidden_size, bias=False),
#     nn.LeakyReLU(.1,inplace=True),
#     #nn.Dropout(.5),
#     nn.Linear(in_features=hidden_size, out_features=1, bias=False),
#     nn.Sigmoid())
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNormAct2d(
      64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): DenseBlock(
      (denselayer1): DenseLayer(
        (norm1): BatchNormAct2d(
          64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNormAct2d(
          128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (a

In [11]:
import numpy as np
np.random.uniform(0,.2)

0.06523762999917541

In [ ]:
#### gc.collect()
import wandb
import timm
import numpy as np

for i in range(10):
    lr = np.random.uniform(3e-5,5e-5)
    epochs = np.random.randint(72,78)
    #epochs=30
    hidden_size = 64
    dropout = .25
    

    model = timm.create_model('regnetx_064', pretrained=True)
#     model.s1.b1.att.channel.act = nn.LeakyReLU(.1)
#     model.s1.b2.att.channel.act = nn.LeakyReLU(.1)
#     model.s2.b1.att.channel.act = nn.LeakyReLU(.1)
#     model.s2.b2.att.channel.act = nn.LeakyReLU(.1)
#     model.s3.b1.att.channel.act = nn.LeakyReLU(.1)
#     model.s3.b2.att.channel.act = nn.LeakyReLU(.1)
#     model.s4.b1.att.channel.act = nn.LeakyReLU(.1)
#     for i in range(3):
#         model.layer1[i].att2.channel.act = nn.LeakyReLU(.1)
#         model.layer2[i].att2.channel.act = nn.LeakyReLU(.1)
#         model.layer3[i].att2.channel.act = nn.LeakyReLU(.1)
#         model.layer4[i].att2.channel.act = nn.LeakyReLU(.1)
# #     for param in model.parameters():
#         param.requires_grad=False

#     for name, param in model.named_parameters():
#         if 'se' in name:
#             param.requires_grad=True

    num_in_features = model.get_classifier().in_features

    # Replace the existing classifier. It's named: classifier
    model.head.fc = nn.Sequential(
        nn.Dropout(dropout),
        nn.Linear(in_features=num_in_features, out_features=hidden_size, bias=False),
        nn.LeakyReLU(.1),
        #nn.Dropout(dropout),
        nn.Linear(in_features=hidden_size, out_features=1, bias=False),
        nn.LeakyReLU(.1),
        nn.Sigmoid())
    torch.cuda.empty_cache() 

    device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    if torch.cuda.is_available():
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.benchmark = True

    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=.0001)
    scheduler=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 600, T_mult=2, eta_min=1e-12, last_epoch=- 1, verbose=False)

    scheduler1 = StepLR(optimizer, step_size=2000000, gamma=0.1)

    criterion = nn.BCELoss()

    test_size=len(test_dataset)
    dataloaders = {'train' : train_dataloader, 'val' : test_dataloader}
    dataset_sizes = {'train': train_size, 'val' : test_size}
    checkpoint = {'model': 'abc',
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict(),
                 'threshold' : 0.5}
    
    wandb.init(
            # set the wandb project where this run will be logged
            project="Model Testing",
            # track hyperparameters and run metadata
            config={
            "learning_rate": lr,
            "architecture": 'GEregnetx_064',
            "dataset": "600x1000-v6-mass",
            "epochs": epochs,
            "batch size": batch_size,
            "hidden size": hidden_size, 
            "optimizer_name": 'Adam',
            "dropout_rate": dropout,
            "weight decay": 1e-4,       
            }
        )
    
    model, train_metrics, val_metrics, sched_steps = train_model(model, criterion, optimizer, [scheduler,scheduler1], num_epochs=epochs)
#     gc.collect()
#     torch.cuda.empty_cache()
#     optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=.01)
#     scheduler=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 1400, T_mult=3, eta_min=1e-12, last_epoch=- 1, verbose=False)
#     model, train_metrics, val_metrics, sched_steps = train_model(model, criterion, optimizer, [scheduler,scheduler1], num_epochs=12)

    test_metrics = test_model(model, criterion,test_dataloader, threshold = .4)
    wandb.finish() 

    gc.collect()
    torch.cuda.empty_cache()  

    PATH='/home/mbadhan/Desktop/mberghouse/pytorch_models/weights/4_16/densenet161_mass_600x1000_run'+str(i)+'_weights'
    torch.save(model.state_dict(), PATH)
    PATH='/home/mbadhan/Desktop/mberghouse/pytorch_models/models/4_16/densenet161_mass_600x1000_run'+str(i)+'_model'
    torch.save(model, PATH)

# gc.collect()
# torch.cuda.empty_cache() 

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr=6e-5, weight_decay=.001)
# scheduler=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 1400, T_mult=3, eta_min=1e-12, last_epoch=- 1, verbose=False)

model, train_metrics, val_metrics, sched_steps = train_model(model, criterion, optimizer, [scheduler,scheduler1], num_epochs=12)


In [ ]:
test_metrics = test_model(model, criterion,test_dataloader, threshold = .4)
wandb.finish() 

In [ ]:
model, train_metrics, val_metrics, sched_steps = train_model(model, criterion, optimizer, [scheduler,scheduler1], num_epochs=26)


In [ ]:
test_metrics = test_model(model, criterion,test_dataloader, threshold = .4)
wandb.finish()

In [ ]:
PATH='/home/mbadhan/Desktop/mberghouse/pytorch_models/weights/final/SSLresnet50_calc_RMS_run2_weights'
torch.save(model.state_dict(), PATH)
PATH='/home/mbadhan/Desktop/mberghouse/pytorch_models/models/final/SSLresnet50_calc_RMS_run2_model'
torch.save(model, PATH)

In [ ]:
#efficientnetv2_s
#ecaresnext50t_32x4d
#ecaresnet50d
#ecaresnet50t
#ecaresnet101d
#efficientnet_b3_gn
#efficientnet_b3
# 'halo2botnet50ts_256',
# , 'halonet26t',
# , 'halonet50ts',
# , 'halonet_h1',
# , 'haloregnetz_b',
#legacy_seresnet50
#mobilenetv3_large_075
#mobilenetv3_small_100
#seresnet50
#seresnext50_32x4d
#skresnet50
#ssl_resnet50
#swsl_resnet50
#visformer_small
#tv_resnet50
#resnest50d  #split Attention
#regnetx_040
#twins_pcpvt_base
#twins_svt_base

### Crossvit Models ###
#  'crossvit_9_240',
# , 'crossvit_9_dagger_240',
# , 'crossvit_15_240',
# , 'crossvit_15_dagger_240',
# , 'crossvit_15_dagger_408',
# , 'crossvit_18_240',
# , 'crossvit_18_dagger_240',
# , 'crossvit_18_dagger_408',
# , 'crossvit_base_240',
# , 'crossvit_small_240',
# , 'crossvit_tiny_240',

### Swin Models ###
# , 'swin_base_patch4_window7_224',
# , 'swin_base_patch4_window7_224_in22k',
# , 'swin_base_patch4_window12_384',
# , 'swin_base_patch4_window12_384_in22k',
# , 'swin_large_patch4_window7_224',
# , 'swin_large_patch4_window7_224_in22k',
# , 'swin_large_patch4_window12_384',
# , 'swin_large_patch4_window12_384_in22k',
# , 'swin_s3_base_224',
# , 'swin_s3_small_224',
# , 'swin_s3_tiny_224',
# , 'swin_small_patch4_window7_224',
# , 'swin_tiny_patch4_window7_224',
# , 'swinv2_base_window8_256',
# , 'swinv2_base_window12_192_22k',
# , 'swinv2_base_window12to16_192to256_22kft1k',
# , 'swinv2_base_window12to24_192to384_22kft1k',
# , 'swinv2_base_window16_256',
# , 'swinv2_cr_base_224',
# , 'swinv2_cr_base_384',
# , 'swinv2_cr_base_ns_224',
# , 'swinv2_cr_giant_224',
# , 'swinv2_cr_giant_384',
# , 'swinv2_cr_huge_224',
# , 'swinv2_cr_huge_384',
# , 'swinv2_cr_large_224',
# , 'swinv2_cr_large_384',
# , 'swinv2_cr_small_224',
# , 'swinv2_cr_small_384',
# , 'swinv2_cr_small_ns_224',
# , 'swinv2_cr_tiny_224',
# , 'swinv2_cr_tiny_384',
# , 'swinv2_cr_tiny_ns_224',
# , 'swinv2_large_window12_192_22k',
# , 'swinv2_large_window12to16_192to256_22kft1k',
# , 'swinv2_large_window12to24_192to384_22kft1k',
# , 'swinv2_small_window8_256',
# , 'swinv2_small_window16_256',
# , 'swinv2_tiny_window8_256',
# , 'swinv2_tiny_window16_256',
###############################